In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [ ]:
class CelebAHQDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.images[idx])
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

dataset = CelebAHQDataset('../../data/celeba_hq_256', transform=transform)

In [ ]:
import torch
import torch.nn as nn

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1), # 128x128
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1), # 64x64
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1), # 32x32
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1), # 16x16
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(256 * 16 * 16, latent_dim),
            nn.ReLU(),
        )

        # Latent space
        self.fc_mu = nn.Linear(latent_dim, latent_dim)
        self.fc_logvar = nn.Linear(latent_dim, latent_dim)
        self.fc_decode = nn.Linear(latent_dim, 256 * 16 * 16)

        # Decoder
        self.decoder = nn.Sequential(
            nn.Unflatten(1, (256, 16, 16)),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1), # 32x32
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1), # 64x64
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1), # 128x128
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, kernel_size=4, stride=2, padding=1), # 256x256
            nn.Sigmoid()
        )

    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = self.fc_decode(z)
        return self.decoder(h)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


In [ ]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

In [ ]:
# 체크포인트 저장 및 로드 함수
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location="cpu")
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

    # Learning rate 재설정
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    
    return checkpoint  # checkpoint 반환

In [ ]:
torch.manual_seed(975)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
latent_dim = 100  # 잠재 공간의 차원
batch_size = 64
lr = 0.001
epochs = 150

In [ ]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
data_loader = DataLoader(dataset, batch_size=64, shuffle=True)

In [ ]:
# 체크포인트 파일명
checkpoint_file = "my_checkpoint.pth.tar"

# 체크포인트가 있으면 로드하고, 없으면 새로 시작
start_epoch = 0
try:
    checkpoint = load_checkpoint(checkpoint_file, model, optimizer, lr=0.001)
    start_epoch = checkpoint['epoch'] + 1
    print(f"Resuming training from epoch {start_epoch}")
except FileNotFoundError:
    print("No checkpoint found, starting from scratch")

In [ ]:
for epoch in tqdm(range(start_epoch, epochs)):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(data_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(data_loader.dataset)
    print(f'Epoch [{epoch+1}/{epochs}], Average Loss: {train_loss:.4f}')

    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
        "epoch": epoch
    }
    save_checkpoint(checkpoint, filename=checkpoint_file)


In [ ]:
def show_images(model):
    model.eval()
    with torch.no_grad():
        z = torch.randn(64, latent_dim).to(device)
        generated_images = model.decode(z)

        fig, axes = plt.subplots(8, 8, figsize=(9, 9))
        for i, ax in enumerate(axes.flat):
            img = generated_images[i].cpu().numpy().transpose(1, 2, 0)
            img = np.clip(img, 0, 1)
            ax.imshow(img)
            ax.axis('off')

        plt.subplots_adjust(wspace=0, hspace=0)
        plt.show()

# 이미지 생성 및 격자 형태로 표시
show_images(model)

In [ ]:
# def show_images(model):
#     model.eval()
#     with torch.no_grad():
#         z = torch.randn(64, latent_dim).to(device)
#         generated_images = model.decode(z)

#         fig, axes = plt.subplots(8, 8, figsize=(9, 9))
#         for i, ax in enumerate(axes.flat):
#             img = generated_images[i].cpu().numpy().transpose(1, 2, 0)
#             img = np.clip(img, 0, 1)
#             ax.imshow(img)
#             ax.axis('off')

#         plt.subplots_adjust(wspace=0, hspace=0)
#         plt.show()

# for epoch in range(epochs):
#     model.train()
#     train_loss = 0
#     for batch_idx, data in enumerate(data_loader):
#         data = data.to(device)
#         optimizer.zero_grad()
#         recon_batch, mu, logvar = model(data)
#         loss = loss_function(recon_batch, data, mu, logvar)
#         loss.backward()
#         train_loss += loss.item()
#         optimizer.step()
        
#         if batch_idx % log_interval == 0:
#             print(f'Epoch [{epoch+1}/{epochs}], Batch {batch_idx}, Loss: {loss.item():.4f}')
            
#         if batch_idx % log_interval == 0:
#             show_images(model)

#     train_loss /= len(data_loader.dataset)
#     print(f'Epoch [{epoch+1}/{epochs}], Average Loss: {train_loss:.4f}')